In [ ]:
from IPython.core.display import HTML
with open ("style.css", "r") as file:
    css = file.read()
html = f"<style>{css}</style>"
HTML(html)

# Checking the Equivalence of Regular Expressions

In order to check whether two regular expressions $r_1$ and $r_2$ are equivalent, perform the 
following steps:
- convert $r_1$ and $r_2$ into non-deterministic <span style="font-variant:small-caps;">Fsm</span>s
  $F_1$ and $F_2$ such that $L(r_1) = L(F_1)$ and $L(r_2) = L(F_2)$,
- convert $F_1$ and $F_2$ into deterministic <span style="font-variant:small-caps;">Fsm</span>s
  $D_1$ and $D_2$ such that $L(D_1) = L(F_1)$ and $L(D_2) = L(F_2)$
- check whether both $L(D_1) \backslash L(D_2)$ and $L(D_2) \backslash L(D_1)$ are empty.

`Regexp-2-NFA.ipynb` contains the function `RegExp2NFA.toNFA` that can be used to compute a non-deterministic 
<span style="font-variant:small-caps;">Fsm</span> that accepts the language described by a given regular expression.

In [ ]:
%run Regexp-2-NFA.ipynb

`NFA-2-DFA.ipynb` contains the function `nfa2dfa` that converts a non-deterministic 
<span style="font-variant:small-caps;">Fsm</span> into an equivalent deterministic 
<span style="font-variant:small-caps;">Fsm</span>.

In [ ]:
%run NFA-2-DFA.ipynb

Given two sets `A` and `B`, the function `cartesian_product(A, B)` computes the 
<em style="color:blue">cartesian product</em> $A \times B$ which is defined as
$$ A \times B := \{ (x, y) \mid x \in A \wedge y \in B \}. $$

In [ ]:
def cartesian_product(A, B):
    return { (x, y) for x in A
                    for y in B
           }

In [ ]:
cartesian_product({1, 2}, {'a', 'b'})

Given to deterministic <span style="font-variant:small-caps;">Fsm</span>s `F1` and `F2`,
the expression `fsm_complement(F1, F2)` computes a deterministic 
<span style="font-variant:small-caps;">Fsm</span> that recognizes the language  $L(F_1)\backslash L(F_2)$.

In [ ]:
def fsm_complement(F1, F2):
    States1, Sigma, delta1, q1, A1 = F1
    States2,     _, delta2, q2, A2 = F2
    States = cartesian_product(States1, States2)
    delta  = {}
    for (p1, p2) in States:
        for c in Sigma:
            delta[(p1, p2), c] = (delta1[p1, c], delta2[p2, c])
    return (States, Sigma, delta, (q1, q2), cartesian_product(A1, States2 - A2))

Given a regular expression $r$ and an alphabet $\Sigma$, the function $\texttt{regexp2DFA}(r, \Sigma)$
computes a deterministic <span style="font-variant:small-caps;">Fsm</span>s that accepts
the language specified by $r$.

In [ ]:
def regexp2DFA(r, Σ):
    converter = RegExp2NFA(Σ)
    States, Σ, δ, q0, A = converter.toNFA(r)
    return nfa2dfa((States, Σ, δ, q0, A))

Given a deterministic <span style="font-variant:small-caps;">Fsm</span> $F$ the function 
`is_empty(F)` checks whether the language accepted by $F$ is empty.
In this function, the variable `Reachable` is the set of those states that are already known to be reachable
from the start state `q0`. `NewFound` are those states that can be reached from a state in the set 
`Reachable`.  When we find no new states that are reachable, the iteration stops and we check whether
there is a state that is both reachable and acceptable because in that case the language is not empty.

In [ ]:
def is_empty(F):
    States, Σ, δ, q0, Accepting = F
    Reachable = { q0 }
    NewFound  = { q0 }
    while True:
        NewFound = { δ[q, c] for q in NewFound
                             for c in Σ 
                   }
        if NewFound <= Reachable:
            break
        Reachable |= NewFound
    return Reachable & Accepting == set()

In [ ]:
%run FSM-2-Dot.ipynb

The function `regExpEquiv` takes three arguments:
- $r_1$ and $r_2$ are regular expressions,
- $\Sigma$ is the alphabet used in these regular expressions.

The function returns `True` iff $r_1 \doteq r_2$, i.e. if $r_1$ and $r_2$ are equivalent. 

In [ ]:
def regExpEquiv(r1, r2, Sigma):
    F1 = regexp2DFA(r1, Sigma)
    F2 = regexp2DFA(r2, Sigma)    
    r1_minus_r2 = fsm_complement(F1, F2)
    r2_minus_r1 = fsm_complement(F2, F1)
    return is_empty(r1_minus_r2) and is_empty(r2_minus_r1)

The notebook `Test-Equivalence.ipynb` can be used to test this function.